In [1]:
import kfp, os
import kfp.dsl as dsl 
import kfp.compiler as compiler
import kubernetes.client.models as k8s
from kfp.aws import use_aws_secret
import namesgenerator
from utils import *

In [2]:
NAMESPACE = os.environ.get("NAMESPACE") 

kubeflow_address = f"http://{NAMESPACE}.kubeflow.odsc.k8s.hydrosphere.io"
hydrosphere_address = f"http://{NAMESPACE}.serving.odsc.k8s.hydrosphere.io"

In [3]:
print(kubeflow_address)
print(hydrosphere_address)

http://df3b0da4.kubeflow.odsc.k8s.hydrosphere.io
http://df3b0da4.serving.odsc.k8s.hydrosphere.io


# 01 Download

### Pipeline

At this stage we will obtain all training data for our pipeline.

In [4]:
username = "hydrosphere"
version = "v1"

In [5]:
def download_op(**kwargs):
    return dsl.ContainerOp(
        name="download",
        image=f"{username}/mnist-pipeline-download:{version}",
        file_outputs={"data_path": "/data_path.txt"},
        arguments=["--hydrosphere-address", kwargs["hydrosphere_address"]]
    ).apply(use_aws_secret())

Create pipeline definition.

In [6]:
@dsl.pipeline(name="mnist", description="MNIST classifier")
def pipeline_definition(hydrosphere_address):
    download = download_op(
        hydrosphere_address=hydrosphere_address)

Compile pipeline.

In [7]:
compiler.Compiler().compile(pipeline_definition, "pipeline.tar.gz")

Replace compiled hardcoded namespaces.

In [9]:
%%bash 

tar -xvf pipeline.tar.gz
sed -i "s/minio-service.kubeflow/minio-service.${NAMESPACE}/g" pipeline.yaml
sed -i "s/pipeline-runner/${NAMESPACE}-pipeline-runner/g" pipeline.yaml

x pipeline.yaml


### Test Run

In [10]:
# Create Pipelines client
client = kfp.Client(kubeflow_address)

In [11]:
# Define an experiment name
experiment_name='MNIST Showreal'

In [12]:
# get or create an experiment_id
try:
    experiment_id = client.get_experiment(experiment_name=experiment_name).id
except:
    experiment_id = client.create_experiment(experiment_name).id

In [13]:
# start a run
run_name = namesgenerator.get_random_name()
print("Starting a new run with the name {}".format(run_name))
result = client.run_pipeline(
    experiment_id, run_name, "pipeline.yaml",
    {
        "hydrosphere-address": hydrosphere_address,
    }
)

Starting a new run with the name agitated_wiles


# 02 Train

### Pipeline

At this stage we will create a model & train it on the downloaded data.

In [14]:
def train_op(**kwargs):
    return dsl.ContainerOp(
        name="train",
        image=f"{username}/mnist-pipeline-train:{version}",
        file_outputs={
            "accuracy": "/accuracy.txt",
            "model_path": "/model_path.txt"
        },
        arguments=[
            "--data-path", kwargs["data_path"],
            "--learning-rate", kwargs["learning_rate"],
            "--epochs", kwargs["epochs"],
            "--batch-size", kwargs["batch_size"],
            "--hydrosphere-address", kwargs["hydrosphere_address"]
        ],
    ).apply(use_aws_secret())

Update pipeline definition.

In [15]:
@dsl.pipeline(name="mnist", description="MNIST classifier")
def pipeline_definition(
    hydrosphere_address,
    learning_rate,
    epochs,
    batch_size,
):
    download = download_op(
        hydrosphere_address=hydrosphere_address)

    train = train_op(
        hydrosphere_address=hydrosphere_address,
        data_path=download.outputs["data_path"], 
        learning_rate=learning_rate,
        epochs=epochs,
        batch_size=batch_size)
    train.after(download)
    train.set_memory_request('1G')
    train.set_cpu_request('1')

Compile pipeline.

In [16]:
compiler.Compiler().compile(pipeline_definition, "pipeline.tar.gz")

Replace compiled hardcoded namespaces.

In [ ]:
%%bash 

tar -xvf pipeline.tar.gz
sed -i "s/minio-service.kubeflow/minio-service.${NAMESPACE}/g" pipeline.yaml
sed -i "s/pipeline-runner/${NAMESPACE}-pipeline-runner/g" pipeline.yaml

### Test Run

In [18]:
# start a run
run_name = namesgenerator.get_random_name()
print("Starting a new run with the name {}".format(run_name))
result = client.run_pipeline(
    experiment_id, run_name, "pipeline.yaml",
    {
        "hydrosphere-address": hydrosphere_address,
        "learning-rate": "0.01",
        "batch-size": "256",
        "epochs": "10"
    }
)

Starting a new run with the name hopeful_ardinghelli


# 03 Release

### Pipeline

At this stage we will release the trained model to the cluster

In [19]:
def release_op(**kwargs):
    return dsl.ContainerOp(
        name="release",
        image=f"{username}/mnist-pipeline-release:{version}",
        file_outputs={"model_version": "/model_version.txt"},
        arguments=[
            "--data-path", kwargs["data_path"],
            "--model-name", kwargs["model_name"],
            "--models-path", kwargs["models_path"],
            "--accuracy", kwargs["accuracy"], 
            "--hydrosphere-address", kwargs["hydrosphere_address"],
            "--learning-rate", kwargs["learning_rate"],
            "--epochs", kwargs["epochs"],
            "--batch-size", kwargs["batch_size"],
        ],
    ).apply(use_aws_secret())

Update pipeline definition.

In [20]:
@dsl.pipeline(name="mnist", description="MNIST classifier")
def pipeline_definition(
    hydrosphere_address,
    learning_rate,
    epochs,
    batch_size,
    model_name,
):
    
    download = download_op(
        hydrosphere_address=hydrosphere_address)

    train = train_op(
        hydrosphere_address=hydrosphere_address,
        data_path=download.outputs["data_path"], 
        learning_rate=learning_rate,
        epochs=epochs,
        batch_size=batch_size)
    train.after(download)
    train.set_memory_request('1G')
    train.set_cpu_request('1')
    
    release = release_op(
        data_path=download.outputs["data_path"],
        models_path=train.outputs["model_path"],
        accuracy=train.outputs["accuracy"],
        model_name=model_name,
        hydrosphere_address=hydrosphere_address,
        learning_rate=learning_rate,
        epochs=epochs,
        batch_size=batch_size)
    release.after(train)

Compile pipeline.

In [21]:
compiler.Compiler().compile(pipeline_definition, "pipeline.tar.gz")

Replace compiled hardcoded namespaces.

In [22]:
%%bash 

tar -xvf pipeline.tar.gz
sed -i "s/minio-service.kubeflow/minio-service.${NAMESPACE}/g" pipeline.yaml
sed -i "s/pipeline-runner/${NAMESPACE}-pipeline-runner/g" pipeline.yaml

x pipeline.yaml


### Test Run

In [23]:
# start a run
run_name = namesgenerator.get_random_name()
print("Starting a new run with the name {}".format(run_name))
result = client.run_pipeline(
    experiment_id, run_name, "pipeline.yaml",
    {
        "hydrosphere-address": hydrosphere_address,
        "learning-rate": "0.01",
        "batch-size": "256",
        "epochs": "10",
        "model_name": "mnist",
    }
)

Starting a new run with the name infallible_shockley


# 04 Deploy to Stage

### Pipeline

At this stage we are deploying the model to the stage application to run integration tests.

In [24]:
def deploy_to_stage_op(**kwargs):
    return dsl.ContainerOp(
        name="deploy_to_stage",
        image=f"{username}/mnist-pipeline-deploy-to-stage:{version}",
        arguments=[
            "--model-version", kwargs["model_version"],
            "--hydrosphere-address", kwargs["hydrosphere_address"],
            "--model-name", kwargs["model_name"],
        ],
    ).apply(use_aws_secret())

Update pipeline definition.

In [25]:
@dsl.pipeline(name="mnist", description="MNIST classifier")
def pipeline_definition(
    hydrosphere_address,
    learning_rate,
    epochs,
    batch_size,
    model_name,
):
    
    download = download_op(
        hydrosphere_address=hydrosphere_address)

    train = train_op(
        hydrosphere_address=hydrosphere_address,
        data_path=download.outputs["data_path"], 
        learning_rate=learning_rate,
        epochs=epochs,
        batch_size=batch_size)
    train.after(download)
    train.set_memory_request('1G')
    train.set_cpu_request('1')
    
    release = release_op(
        data_path=download.outputs["data_path"],
        models_path=train.outputs["model_path"],
        accuracy=train.outputs["accuracy"],
        model_name=model_name,
        hydrosphere_address=hydrosphere_address,
        learning_rate=learning_rate,
        epochs=epochs,
        batch_size=batch_size)
    release.after(train)
    
    deploy_to_stage = deploy_to_stage_op(
        model_version=release.outputs["model_version"],
        hydrosphere_address=hydrosphere_address,
        model_name=model_name)
    deploy_to_stage.after(release)

Compile pipeline.

In [26]:
compiler.Compiler().compile(pipeline_definition, "pipeline.tar.gz")

Replace compiled hardcoded namespaces.

In [27]:
%%bash 

tar -xvf pipeline.tar.gz
sed -i "s/minio-service.kubeflow/minio-service.${NAMESPACE}/g" pipeline.yaml
sed -i "s/pipeline-runner/${NAMESPACE}-pipeline-runner/g" pipeline.yaml

x pipeline.yaml


### Test Run

In [28]:
# start a run
run_name = namesgenerator.get_random_name()
print("Starting a new run with the name {}".format(run_name))
result = client.run_pipeline(
    experiment_id, run_name, "pipeline.yaml",
    {
        "hydrosphere-address": hydrosphere_address,
        "learning-rate": "0.01",
        "batch-size": "256",
        "epochs": "10",
        "model_name": "mnist",
    }
)

Starting a new run with the name unruffled_torvalds


# 05 Test

### Pipeline

At this stage we are performing integration tests.

In [29]:
def test_op(**kwargs):
    return dsl.ContainerOp(
        name="test",
        image=f"{username}/mnist-pipeline-test:{version}", 
        arguments=[
            "--data-path", kwargs["data_path"],
            "--hydrosphere-address", kwargs["hydrosphere_address"],
            "--acceptable-accuracy", kwargs["acceptable_accuracy"],
            "--model-name", kwargs["model_name"], 
        ],
    ).apply(use_aws_secret())

Update pipeline definition.

In [30]:
@dsl.pipeline(name="mnist", description="MNIST classifier")
def pipeline_definition(
    hydrosphere_address,
    learning_rate,
    epochs,
    batch_size,
    model_name,
    acceptable_accuracy
):
    
    download = download_op(
        hydrosphere_address=hydrosphere_address)

    train = train_op(
        hydrosphere_address=hydrosphere_address,
        data_path=download.outputs["data_path"], 
        learning_rate=learning_rate,
        epochs=epochs,
        batch_size=batch_size)
    train.after(download)
    train.set_memory_request('1G')
    train.set_cpu_request('1')
    
    release = release_op(
        data_path=download.outputs["data_path"],
        models_path=train.outputs["model_path"],
        accuracy=train.outputs["accuracy"],
        model_name=model_name,
        hydrosphere_address=hydrosphere_address,
        learning_rate=learning_rate,
        epochs=epochs,
        batch_size=batch_size)
    release.after(train)
    
    deploy_to_stage = deploy_to_stage_op(
        model_version=release.outputs["model_version"],
        hydrosphere_address=hydrosphere_address,
        model_name=model_name)
    deploy_to_stage.after(release)
    
    test = test_op(
        data_path=download.outputs["data_path"],
        hydrosphere_address=hydrosphere_address,
        acceptable_accuracy=acceptable_accuracy,
        model_name=model_name)
    test.set_retry(3)
    test.after(deploy_to_stage)

Compile pipeline.

In [31]:
compiler.Compiler().compile(pipeline_definition, "pipeline.tar.gz")

Replace compiled hardcoded namespaces.

In [32]:
%%bash 

tar -xvf pipeline.tar.gz
sed -i "s/minio-service.kubeflow/minio-service.${NAMESPACE}/g" pipeline.yaml
sed -i "s/pipeline-runner/${NAMESPACE}-pipeline-runner/g" pipeline.yaml

x pipeline.yaml


### Test Run

In [33]:
# start a run
run_name = namesgenerator.get_random_name()
print("Starting a new run with the name {}".format(run_name))
result = client.run_pipeline(
    experiment_id, run_name, "pipeline.yaml",
    {
        "learning-rate": "0.01",
        "batch-size": "256",
        "epochs": "10",
        "hydrosphere-address": hydrosphere_address,
        "acceptable-accuracy": "0.90",
        "model_name": "mnist",
    }
)

Starting a new run with the name blissful_mclean


# 06 Deploy to Production

### Pipeline

And finally deploying the model to production application

Define container operation

In [34]:
def deploy_to_prod_op(**kwargs):
    return dsl.ContainerOp(
        name="deploy_to_prod",
        image=f"{username}/mnist-pipeline-deploy-to-prod:{version}",
        arguments=[
            "--model-version", kwargs["model_version"],
            "--model-name", kwargs["model_name"],
            "--hydrosphere-address", kwargs["hydrosphere_address"]
        ],
    ).apply(use_aws_secret())

Update pipeline definition.

In [35]:
@dsl.pipeline(name="mnist", description="MNIST classifier")
def pipeline_definition(
    hydrosphere_address,
    learning_rate,
    epochs,
    batch_size,
    model_name,
    acceptable_accuracy
):
    
    download = download_op(
        hydrosphere_address=hydrosphere_address)

    train = train_op(
        hydrosphere_address=hydrosphere_address,
        data_path=download.outputs["data_path"], 
        learning_rate=learning_rate,
        epochs=epochs,
        batch_size=batch_size)
    train.after(download)
    train.set_memory_request('1G')
    train.set_cpu_request('1')
    
    release = release_op(
        data_path=download.outputs["data_path"],
        models_path=train.outputs["model_path"],
        accuracy=train.outputs["accuracy"],
        model_name=model_name,
        hydrosphere_address=hydrosphere_address,
        learning_rate=learning_rate,
        epochs=epochs,
        batch_size=batch_size)
    release.after(train)
    
    deploy_to_stage = deploy_to_stage_op(
        model_version=release.outputs["model_version"],
        hydrosphere_address=hydrosphere_address,
        model_name=model_name)
    deploy_to_stage.after(release)
    
    test = test_op(
        data_path=download.outputs["data_path"],
        hydrosphere_address=hydrosphere_address,
        acceptable_accuracy=acceptable_accuracy,
        model_name=model_name)
    test.set_retry(3)
    test.after(deploy_to_stage)
    
    deploy_to_prod = deploy_to_prod_op(
        model_version=release.outputs["model_version"],
        model_name=model_name,
        hydrosphere_address=hydrosphere_address)
    deploy_to_prod.after(test)

Compile pipeline.

In [36]:
compiler.Compiler().compile(pipeline_definition, "pipeline.tar.gz")

Replace compiled hardcoded namespaces.

In [37]:
%%bash 

tar -xvf pipeline.tar.gz
sed -i "s/minio-service.kubeflow/minio-service.${NAMESPACE}/g" pipeline.yaml
sed -i "s/pipeline-runner/${NAMESPACE}-pipeline-runner/g" pipeline.yaml

x pipeline.yaml


### Test Run

In [38]:
# start a run
run_name = namesgenerator.get_random_name()
print("Starting a new run with the name {}".format(run_name))
result = client.run_pipeline(
    experiment_id, run_name, "pipeline.yaml",
    {
        "learning-rate": "0.01",
        "batch-size": "256",
        "epochs": "10",
        "hydrosphere-address": hydrosphere_address,
        "acceptable-accuracy": "0.90",
        "model_name": "mnist",
    }
)

Starting a new run with the name condescending_varahamihira


## Production traffic

In [ ]:
simulate_production_traffic(shuffle=False)

# 01 Sample

### Pipeline

Once we have a fully functional pipeline, we would like to automate running this pipeline.

In [45]:
def sample_op(**kwargs):
    return dsl.ContainerOp(
        name="sample",
        image=f"{username}/mnist-pipeline-sample:{version}", 
        file_outputs={"data_path": "/data_path.txt"},
        arguments=[
            "--hydrosphere-address", kwargs["hydrosphere_address"],
            "--model-name", kwargs["model_name"],
        ]
    ).apply(use_aws_secret())

Update pipeline definition.

In [50]:
@dsl.pipeline(name="mnist", description="MNIST classifier")
def pipeline_definition(
    hydrosphere_address,
    learning_rate,
    epochs,
    batch_size,
    model_name,
    acceptable_accuracy
):
    
    sample = sample_op(
        hydrosphere_address=hydrosphere_address,
        model_name=model_name)
    
    train = train_op(
        hydrosphere_address=hydrosphere_address,
        data_path=sample.outputs["data_path"], 
        learning_rate=learning_rate,
        epochs=epochs,
        batch_size=batch_size)
    train.after(sample)
    train.set_memory_request('1G')
    train.set_cpu_request('1')
    
    release = release_op(
        data_path=sample.outputs["data_path"],
        models_path=train.outputs["model_path"],
        accuracy=train.outputs["accuracy"],
        model_name=model_name,
        hydrosphere_address=hydrosphere_address,
        learning_rate=learning_rate,
        epochs=epochs,
        batch_size=batch_size)
    release.after(train)
    
    deploy_to_stage = deploy_to_stage_op(
        model_version=release.outputs["model_version"],
        hydrosphere_address=hydrosphere_address,
        model_name=model_name)
    deploy_to_stage.after(release)
    
    test = test_op(
        data_path=sample.outputs["data_path"],
        hydrosphere_address=hydrosphere_address,
        acceptable_accuracy=acceptable_accuracy,
        model_name=model_name)
    test.set_retry(3)
    test.after(deploy_to_stage)
    
    deploy_to_prod = deploy_to_prod_op(
        model_version=release.outputs["model_version"],
        model_name=model_name,
        hydrosphere_address=hydrosphere_address)
    deploy_to_prod.after(test)

Compile pipeline.

In [51]:
compiler.Compiler().compile(pipeline_definition, "pipeline.tar.gz")

Replace compiled hardcoded namespaces.

In [52]:
%%bash 

tar -xvf pipeline.tar.gz
sed -i "s/minio-service.kubeflow/minio-service.${NAMESPACE}/g" pipeline.yaml
sed -i "s/pipeline-runner/${NAMESPACE}-pipeline-runner/g" pipeline.yaml

x pipeline.yaml


### Test Run

In [54]:
# start a run
run_name = namesgenerator.get_random_name()
print("Starting a new run with the name {}".format(run_name))
result = client.run_pipeline(
    experiment_id, run_name, "pipeline.yaml",
    {
        "learning-rate": "0.0005",
        "batch-size": "256",
        "epochs": "100",
        "hydrosphere-address": hydrosphere_address,
        "acceptable-accuracy": "0.60",
        "model_name": "mnist",
    }
)

Starting a new run with the name sharp_spence


## Production traffic

In [ ]:
simulate_production_traffic(shuffle=False)